In [11]:
import requests
from bs4 import BeautifulSoup


res = requests.get('https://www.i.ua/')

soup = BeautifulSoup(res.text, 'lxml')

In [12]:
 # https://www.i.ua/  скріпт для отримання новин
for n in soup.find('ul', id='news_cont').find_all('li'):
    time = n.find('span').text
    href = n.find('a').get('href')
    info = n.find('a').text
    
    print(time, href, info)

19:34 https://news.i.ua/click/7830218/812567310/2 Группа киевлян под видом иностранного обмена переправляла нелегалов в страны ЕС
19:33 https://news.i.ua/click/7830214/842739770/2 Устрашающие и мистические: 10 способов отметить Хэллоуин в Киеве
19:32 https://news.i.ua/click/7830217/1961621420/2 Телескоп Hubble к празднику Хэллоуина показал снимок умирающей звезды
19:31 https://news.i.ua/click/7830205/1382926354/2 В одной из громад Днепропетровщины демонтируют незаконные ограждения
19:31 https://news.i.ua/click/7830206/1131378462/2 Хакеру, создавшему фейковую «Дию», предложили стажировку в Минцифры
19:30 https://news.i.ua/click/7830213/2046309541/2 У Києві сталась пожежа в ресторані: Інформації про постраждалих поки немає
19:27 https://news.i.ua/click/7830201/237233962/2 Приложение Apple Music исправили карту с Крымом
19:23 https://news.i.ua/click/7830192/479425903/2 Назван напиток, который втрое повышает риск инсульта
19:22 https://news.i.ua/click/7830202/1539709957/2 Розробнику фейков

In [23]:
# отримати список випадкових анкет та зібрати з них всю детальну інформацію
link = 'https://love.i.ua/'

DB = {}

res = requests.get(link)

soup = BeautifulSoup(res.text, 'lxml')

In [37]:
for person in soup.find_all('ul', class_='list_cols_5 clear')[1].find_all('li'):
    href = person.find('a').get('href')
    name = person.find('dd').find('a').text
    DB[href] = {}
    DB[href]['name'] = name
DB    

{'/user/2082529/': {'name': 'Юрий'},
 '/user/3363036/': {'name': 'Бюро Строй Услуг'},
 '/user/727686/': {'name': 'Olga'},
 '/user/754836/': {'name': "Мак's"},
 '/user/1540849/': {'name': 'Владимир'}}

In [75]:
from datetime import datetime
for k,v in DB.items():
    res = requests.get(link.rstrip('/')+k)
    soup = BeautifulSoup(res.text, 'lxml')
    v['des'] = soup.find('div', class_="profile_info").find('div', id="name_block").text
    v['search'] = soup.find('div', class_="profile_info").find('div', id="look_block").text
    age = soup.find('div', class_="profile_info").find('ul', class_="list_cols_2 clear").li.b.text.split()[0]
    v['age'] = int(age)
    
    date_add = soup.find('div', class_="profile_info").find('ul', class_="list_cols_2 clear").span.text[-10:]
    v['date_add'] = datetime.strptime(date_add, '%d.%m.%Y')
    
    photo = soup.find('div', class_="profile_photo").img.get('src')
    
    res = requests.get(photo)
    
    with open('photo/'+v['name']+'.jpeg', 'wb') as f:
        f.write(res.content)
    
    

In [64]:
from pprint import pprint

pprint(DB)

{'/user/1540849/': {'age': 36,
                    'date_add': datetime.datetime(2008, 11, 22, 0, 0),
                    'des': 'Умный, добрый',
                    'name': 'Владимир',
                    'search': 'Ищу девушку 21—35 летдевушку без вредных '
                              'привычек'},
 '/user/2082529/': {'age': 44,
                    'date_add': datetime.datetime(2016, 3, 14, 0, 0),
                    'des': 'Стройный образованный, ищу такую же)',
                    'name': 'Юрий',
                    'search': 'Ищу девушку 28—32 летСтройную девушку до 32, '
                              'возраст и наличие детей имеет значение.'},
 '/user/3363036/': {'age': 41,
                    'date_add': datetime.datetime(2012, 12, 8, 0, 0),
                    'des': 'Обычная хотя интересная',
                    'name': 'Бюро Строй Услуг',
                    'search': 'Ищу парня 50—60 летНе пьющего .симпотичного.не '
                              'озабоченого'},
 '/user/7276

In [87]:
# обробка текстового файла
from pprint import pprint
FILE = 'spare_parts.txt'

def get_data(path_to_file):
    """Відкриваємо файл та формуємо із нього dict"""
    
    db = {}
    for row in open(path_to_file, encoding='UTF-8'):
        sku, name, inStock, price = row.split('|')
        price = float(price.rstrip(' грн\n'))
        db[sku] = [name, inStock, price]
        
    return db
        
def qty_part_in_price(db):
    """Виводить на екран к-сть товарів в прайсі"""
    
    qty = len(db)
    print(f'Кількість товарів в прайсі = {qty}')

def qty_part_in_stock(db):
    """Виводить на екран к-сть товарів в прайсі в наявності"""
#     qty = 0
#     for sku,v in db.items():
#         if v[1] == 'В наличии':
#             qty += 1
    
    qty = len(list(filter(lambda sku: db[sku][1] == 'В наличии', db)))
    
    print(f'Кількість товарів в наявності = {qty}')
            

def max_part(db):
    """Ціна та назва найдорожчої товарної позиції"""
    max_price = -1
    name = None
    
    for sku,v in db.items():
        if v[2] > max_price:
            max_price = v[2]
            name = v[0]
    print(f'Найдорожча позиція {name} з ціною = {max_price}')
    

def myPickle(db):
    """Сереалізація даних"""
    
    import pickle
    
    with open('db.pickle', 'wb') as f:
        pickle.dump(db, f)
    
    print('Сереалізація пройшла успішно')

    
def main():
    db = get_data(FILE)
    
    qty_part_in_price(db)
    qty_part_in_stock(db)
    max_part(db)
    
    myPickle(db)
    


if __name__ == '__main__':
    main()

Кількість товарів в прайсі = 426
Кількість товарів в наявності = 353
Найдорожча позиція Масло Liqui Moly TOP TEC 4600 5W-30 5л з ціною = 1623.0
Сереалізація пройшла успішно
